In [1]:
import numpy as np
import cv2
from pathlib import Path
from pathlib import Path
from IPython.display import Image

In [2]:
#pip install numpy

In [3]:
#pip install pandas

In [4]:
#pip install torch

In [5]:
#pip install torch torchvision

In [6]:
#pip install --upgrade setuptools

In [7]:
#pip install opencv-python

In [8]:
#pip install pyttsx3

In [9]:
#pip install torch torchvision torchaudio --upgrade

In [10]:
import torch

In [11]:
model=torch.hub.load('ultralytics/yolov5','yolov5s',pretrained=True)

Using cache found in C:\Users\Shriraksha B S/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-10 Python-3.12.2 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [12]:
import cv2
import pyttsx3
import time  # Import the time module

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Open camera stream
cap = cv2.VideoCapture(0)

# Load your model here (assuming 'model' is defined somewhere in your original context)
# model = ...

while True:
    ret, frame = cap.read()
    
    # Check if frame was successfully captured
    if not ret:
        print("Failed to capture frame from camera.")
        break
    
    # Convert the frame to RGB color space if needed
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Perform object detection
    results = model(frame)
    df_result = results.pandas().xyxy[0]
    dict_result = df_result.to_dict()
    scores = list(dict_result["confidence"].values())
    labels = list(dict_result["name"].values())
    
    list_boxes = []
    for dict_item in df_result.to_dict('records'):
        list_boxes.append(list(dict_item.values())[:4])
    
    # Display detection results and speak out closest object
    closest_object = None
    min_distance = float('inf')
    for i, (xmin, ymin, xmax, ymax) in enumerate(list_boxes):
        distance = (xmax - xmin) * 2 + (ymax - ymin) * 2  # Euclidean distance
        if distance < min_distance:
            min_distance = distance
            closest_object = labels[i]
            
        frame = cv2.rectangle(frame, pt1=(int(xmin), int(ymin)), pt2=(int(xmax), int(ymax)), color=(255, 0, 0), thickness=2)
        cv2.putText(frame, f"{labels[i]}: {round(scores[i], 2)}", (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        
    # Speak out the name of the closest object
    if closest_object is not None:
        engine.say(closest_object)
        engine.runAndWait()
    
    # Convert the frame back to BGR color space for display
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    cv2.imshow('Object Detector', frame)
    
    # Introduce a delay to increase latency
    time.sleep(2.5)  # Adjust the sleep duration as needed (in seconds)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break